In [0]:
from numpy import array
from pickle import load,dump
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import Bidirectional
from keras.layers import *
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
import pydot
from numpy import argmax
from nltk.translate.bleu_score import corpus_bleu

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split(' ')[0]
		dataset.append(identifier)
	return set(dataset)
 
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions
 
# load photo features
def load_photo_features(filename, dataset):
    # load all features
    all_features = load(open(filename, 'rb'))
    #print(all_features)
    # filter features
    features = {k: all_features[k] for k in dataset}
    return features
 
# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc
 
# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 
# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)
 
# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, desc_list, photo):
	X1, X2, y = list(), list(), list()
	# walk through each description for the image
	for desc in desc_list:
		# encode the sequence
		seq = tokenizer.texts_to_sequences([desc])[0]
		# split one sequence into multiple X,y pairs
		for i in range(1, len(seq)):
			# split into input and output pair
			in_seq, out_seq = seq[:i], seq[i]
			# pad input sequence
			in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
			# encode output sequence
			out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
			# store
			X1.append(photo)
			X2.append(in_seq)
			y.append(out_seq)
	return array(X1), array(X2), array(y)
 
#data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, tokenizer, max_length):
	# loop for ever over images
	while 1:
		for key, desc_list in descriptions.items():
			# retrieve the photo feature
			photo = photos[key][0]
			in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo)
			yield [[in_img, in_seq], out_word]

# load training dataset (6K)
filename = 'drive/My Drive/Final Year Project/Descriptions/trainuc.txt'
train = load_set(filename)
#print(sorted(train))
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('drive/My Drive/Final Year Project/Descriptions/trainuc.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
#print(train)
#all_features = load(open('features_inceptionv3_uc.pkl', 'rb'))
#print(all_features)
train_features = load_photo_features('drive/My Drive/Final Year Project/Features/features_uc_resnet152_updated.pkl', train)
print('Photos: train=%d' % len(train_features))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
#dump(tokenizer,open('tokenizer_resnet152.pkl','wb'))
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
## determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)
 

Dataset: 1260
Descriptions: train=1260
Photos: train=1260
Vocabulary Size: 293
Description Length: 24


In [0]:
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
 
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text
 
# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length,filename):
    actual, predicted = list(), list()
    # step over the whole set
    lines = list()
    for key, desc_list in descriptions.items():
        yhat = generate_desc(model, tokenizer, photos[key], max_length)
        ex=yhat
        a=yhat.split('startseq')
        b=a[1].split('endseq')
        lines.append('beam_size_1'+'\t'+key + '\t' + b[0])
        references = [d.split() for d in desc_list]
        actual.append(references)
        predicted.append(yhat.split())
        #
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()
    bleu=np.zeros(4)
    # calculate BLEU score
    bleu[0]=corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0))
    bleu[1]=corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0))
    bleu[2]=corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0))
    bleu[3]=corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25))
    print('BLEU-1: %f' % bleu[0])
    print('BLEU-2: %f' % bleu[1])
    print('BLEU-3: %f' % bleu[2])
    print('BLEU-4: %f' % bleu[3])
    return bleu

In [0]:
# load test set
filename = 'drive/My Drive/Final Year Project/Descriptions/testall.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions('drive/My Drive/Final Year Project/Descriptions/testall.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('drive/My Drive/Final Year Project/Features/features_resnet152_combined.pkl', test)
print('Photos: test=%d' % len(test_features))

Dataset: 705
Descriptions: test=705
Photos: test=705


In [0]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints


class Attention(Layer):
    def __init__(self, step_dim=max_length,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim


In [0]:
# load a word embedding
def load_embedding(tokenizer, vocab_size, max_length):
	# load the tokenizer
	embedding = load(open('UCMDataset1/Features/word2vec_embedding.pkl', 'rb'))
	dimensions = 100
	trainable = False
	# create a weight matrix for words in training docs
	weights = np.zeros((vocab_size, dimensions))
	# walk words in order of tokenizer vocab to ensure vectors are in the right index
	for word, i in tokenizer.word_index.items():
		if word not in embedding:
			continue
		weights[i] = embedding[word]
	layer = Embedding(vocab_size, dimensions, weights=[weights], input_length=max_length, trainable=trainable, mask_zero=True)
	return layer

In [0]:
loc = 'drive/My Drive/Final Year Project/Modeluc/'
# define the captioning model
def define_model(vocab_size, max_length):
    # feature extractor model
    inputs1 = Input(shape=(2048,))
    #fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(inputs1)
    fe3 = RepeatVector(max_length)(fe2)
    # sequence model
    inputs2 = Input(shape=(max_length,))
    #se1 = load_embedding(tokenizer, vocab_size, max_length)(inputs2)
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    #print(se1.shape)
    se2 = LSTM(256,return_sequences=True)(se1)
    #print(se2.shape)
    #x = Attention(max_length)(se2) 
    #print(x.shape)
    se3 = TimeDistributed(Dense(256,activation='relu'))(se2)
    #print(se3.shape)
    # decoder model
    decoder1 = concatenate([fe3, se3])
    decoder2 = Bidirectional(LSTM(150,return_sequences=True))(decoder1)
    decoder3 = Attention(max_length)(decoder2)
    outputs = Dense(vocab_size, activation='softmax')(decoder3)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    # summarize model
    model.summary()
    plot_model(model, to_file=loc+'model.png', show_shapes=True)
    return model

In [0]:
model = define_model(vocab_size, max_length)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 24)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 24, 256)      75008       input_4[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 256)          524544      input_3[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LS

In [0]:
## train the model, run epochs manually and save after each epoch
epochs = 20
steps = len(train_descriptions)
mylist = list()
df=pd.DataFrame(index=mylist, columns=['model_no','Bleu_1','Bleu_2','Bleu_3','Bleu_4'])
for i in range(1,epochs):
    print(i)
    # create the data generator
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    # fit for one epoch
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    # save model
    model.save(loc+'model_' + str(i) + '.h5')

1
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
1260/1260 [==============================] - 192s 152ms/step - loss: 3.0008
2
Epoch 1/1
1260/1260 [==============================] - 186s 148ms/step - loss: 1.4319
3
Epoch 1/1
1260/1260 [==============================] - 184s 146ms/step - loss: 0.7636
4
Epoch 1/1
1260/1260 [==============================] - 185s 147ms/step - loss: 0.5542
5
Epoch 1/1
1260/1260 [==============================] - 184s 146ms/step - loss: 0.4650
6
Epoch 1/1
1260/1260 [==============================] - 182s 145ms/step - loss: 0.4128
7
Epoch 1/1
1260/1260 [==============================] - 184s 146ms/step - loss: 0.3710
8
Epoch 1/1
1260/1260 [==============================] - 182s 145ms/step - loss: 0.3377
9
Epoch 1/1
1260/1260 [==============================] - 183s 145ms/step - loss: 0.3086
10
Epoch 1/1
1260/1260 [==============================] - 183s 145ms/step - loss: 0.2911
11
Epoch 1/1
1260/1260 [==============================] - 185s 147ms/st